Extracting all the borders and storing them (+ reversed version) in a dictionary.  
Assuming the corners don't match the other borders

In [1]:
from collections import defaultdict, Counter
from math import sqrt
from copy import deepcopy

In [2]:
data = open("input/20").read()

In [3]:
tiles = defaultdict(dict)
tile_size = 0 # Width and height the same
for tile in data.split("\n\n"):
    num = int(tile.split(":")[0][5:])
    tmp = []
    for r, row in enumerate(tile.split("\n")[1:]):
        row_ = []
        for c, elem in enumerate(row):
            row_.append(elem)
            tile_size = max(tile_size, r)
        tmp.append(row)
    tiles[num] = tmp

In [4]:
edge_origin = defaultdict(list)

def add_stuff(key, string):
    edge_origin[string].append(key)
    edge_origin[string[::-1]].append(key)
    
for key, g in tiles.items():
    top_row = "".join([g[0][c] for c in range(tile_size + 1)])
    add_stuff(key, top_row)

    bottom_row = "".join([g[tile_size][c] for c in range(tile_size + 1)])
    add_stuff(key, bottom_row)

    first_col = "".join([g[r][0] for r in range(tile_size + 1)])
    add_stuff(key, first_col)

    last_col = "".join([g[r][tile_size] for r in range(tile_size + 1)])
    add_stuff(key, last_col)

In [5]:
nums = []
for key, values in edge_origin.items():
    if len(values) == 1:
        nums.append(values[0])

In [6]:
part1 = 1
corners = []
for k, v in Counter(nums).items():
    if v == 4:
        corners.append(k)
        part1 *= k

In [7]:
print(f"Answer #1: {part1}")

Answer #1: 15003787688423


# Part 2
Trying to find the corners + edges and how they fit together

In [8]:
edges = set()
for key, values in edge_origin.items():
    if len(values) == 1:
        if values[0] not in corners:
            edges.add(values[0]) 

In [9]:
middle = []
for key in tiles.keys():
    if key in corners:
        continue
    if key in edges:
        continue
    middle.append(key)

In [10]:
neighbours = defaultdict(set)
for k, v in edge_origin.items():
    if len(v) == 1:
        continue
    neighbours[v[0]].add(v[1])
    neighbours[v[1]].add(v[0])

In [11]:
# Create the tile order first, going around the border and then all in the middle

In [12]:
tile_order = {}
image_size = int(sqrt(len(tiles)))
for x in range(image_size):
    for y in range(image_size):
        tile_order[(x, y)] = None
tile_order[(0, 0)] = corners[0]

In [13]:
def filter_out(ns):
    # Ignore the ones already taken + the one in the middle
    for t in list(tile_order.values()) + middle:
        if t in ns:
            ns.remove(t)

In [14]:
# First row
for x in range(1, image_size):
    ns = list(neighbours[tile_order[(0, x - 1)]])
    filter_out(ns)
    tile_order[(0, x)] = ns[0]

# First column
for y in range(1, image_size):
    ns = list(neighbours[tile_order[(y - 1, 0)]])
    filter_out(ns)
    tile_order[(y, 0)] = ns[0]

# Last row
for x in range(1, image_size):
    ns = list(neighbours[tile_order[(image_size - 1, x - 1)]])
    filter_out(ns)
    tile_order[(image_size - 1, x)] = ns[0]

# Last column
for y in range(1, image_size - 1):  # We've already placed the last elem from 'Last row'
    ns = list(neighbours[tile_order[(y - 1, image_size - 1)]])
    filter_out(ns)
    tile_order[(y, image_size - 1)] = ns[0]

In [15]:
reversed_tile_order = {v: k for k, v in tile_order.items()}

In [16]:
# Fill in the ones in the middle
# Get all neighbours, if we have placed at least two of them we can place it
# Get all possible locations (up, down, left, right) from neighbours and get the most common one, should only be one
while None in tile_order.values():
    for t in middle:
        if t in reversed_tile_order:
            continue
        picked_ns = [n for n in list(neighbours[t]) if n in reversed_tile_order]
       
        # If we have two we can place it
        if len(picked_ns) < 2:
            continue

        possible_locations = defaultdict(int)
        for n in picked_ns:
            x1, y1 = reversed_tile_order[n]
            for x2, y2 in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
                possible_locations[(x1 + x2, y1 + y2)] += 1
        for pos, count in possible_locations.items():
            if tile_order.get(pos) is not None:  # Ignore locations where we already placed a tile
                continue
            if count == len(picked_ns):
                my_pos = pos
        tile_order[my_pos] = t
        reversed_tile_order[t] = my_pos

In [17]:
# Now rotate...
# Reusing the matrix rotation stuff I did in 2017-21

In [18]:
def rotate_90(matrix):
    return [list(row) for row in zip(*matrix[::-1])]
def flip_horizontal(matrix):
    return [row[::-1] for row in matrix]
def flip_vertical(matrix):
    return matrix[::-1]

In [19]:
def flip_and_rotate(initial):
    all_versions = set()
    original = [tuple(row) for row in initial]
    all_versions.add(tuple(original))
    for _ in range(4):
        original = rotate_90(original)
        all_versions.add(tuple([tuple(r) for r in original]))

    for version in deepcopy(all_versions):
        flipped_horizontal = flip_horizontal(version)
        all_versions.add(tuple([tuple(r) for r in flipped_horizontal]))

        flipped_vertical = flip_vertical(version)
        all_versions.add(tuple([tuple(r) for r in flipped_vertical]))

    return all_versions

### Approach
Started with the the top left and the one to the right of that  
generate all combinations (8 x 8) and then from there just assume they only fit in one way

Didn't work, they fit in two ways. Need to include more tiles...

It works by always checking four tiles

A B
C D

Rotate those four in all 8 possible combinations and compare together (4096 combinations but it's fast so won't optimize)

I tried and all only works in one way. So align all four and save when we're done

In [20]:
def get_first_row(matrix):
    return "".join(matrix[0])

def get_last_row(matrix):
    return "".join(matrix[-1])

def get_first_col(matrix):
    col = ""
    for row in matrix:
        col += row[0]
    return col

def get_last_col(matrix):
    col = ""
    for row in matrix:
        col += row[-1]
    return col

In [21]:
structured_tiles = {}

In [22]:
def rotate_and_match(x, y):
    if (x + 1, y + 1) not in tile_order: # Will go outside grid
        return
    first = flip_and_rotate(tiles[tile_order[(x, y)]])
    second = flip_and_rotate(tiles[tile_order[(x, y + 1)]])
    third = flip_and_rotate(tiles[tile_order[(x + 1, y)]])
    fourth = flip_and_rotate(tiles[tile_order[(x + 1, y + 1)]])

    for f in first:
        f_last_col = get_last_col(f)
        f_last_row = get_last_row(f)
        
        for s in second:
            s_first_col = get_first_col(s)
            s_last_row = get_last_row(s)
    
            for t in third:
                t_first_row = get_first_row(t)
                t_last_col = get_last_col(t)
    
                for fo in fourth:
                    fo_first_col = get_first_col(fo)
                    fo_first_row = get_first_row(fo)
                    
                    if f_last_col != s_first_col:
                        continue
    
                    if f_last_row != t_first_row:
                        continue
    
                    if s_last_row != fo_first_row:
                        continue
    
                    if t_last_col != fo_first_col:
                        continue
                    
                    # This was evaluated to always be 1 combinations reaching here.
                    # And also checked that another set of 4 doesn't give different answer...
                    if (x, y) not in structured_tiles:
                        structured_tiles[(x, y)] = f
                    if (x, y + 1) not in structured_tiles:
                        structured_tiles[(x, y + 1)] = s
                    if (x + 1, y) not in structured_tiles:
                        structured_tiles[(x + 1, y)] = t
                    if (x + 1, y + 1) not in structured_tiles:
                        structured_tiles[(x + 1, y + 1)] = fo

In [23]:
for pos in tile_order.keys():
    rotate_and_match(*pos)    

### Remove the borders around the tiles

In [24]:
no_border_tiles = {}
for key, tile in structured_tiles.items():
    new_tile = []
    for row in tile[1:-1]:
        new_tile.append(row[1:-1])
    no_border_tiles[key] = new_tile

In [25]:
new_tile_size = tile_size - 1 # removing 2 borders and adding 1 (0 indexed)
image = []
for r in range(new_tile_size * image_size):
    tmp = []
    for c in range(new_tile_size * image_size):
        tmp.append(None)
    image.append(tmp)

In [26]:
# Copy to big image
for pos, tile in no_border_tiles.items():
    x, y = pos
    for r, row in enumerate(tile):
        for c, elem in enumerate(row):
            image[y * new_tile_size + c][x * new_tile_size + r] = elem

In [27]:
sea_monster = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """

In [28]:
sea_monster_coords = []
for r, row in enumerate(sea_monster.split("\n")):
    for c, elem in enumerate(row):
        if elem == "#":
            sea_monster_coords.append((r, c))
sea_monster_size = len(sea_monster_coords)

In [29]:
def find_sea_monster(matrix):
    num_monsters = 0
    for r in range(len(matrix)):
        for c in range(len(matrix[0])):
            valid = 0
            for rr, cc in sea_monster_coords:
                if r + rr >= len(matrix):
                    break
                if c + cc >= len(matrix[0]):
                    break
                if matrix[c + cc][r + rr] == "#":
                    valid += 1
            if valid == sea_monster_size:
                num_monsters += 1
    return num_monsters    

In [30]:
for idx, img in enumerate(flip_and_rotate(image)):
    num_monsters = find_sea_monster(img)
    if num_monsters != 0:
        break

In [31]:
num_squares = 0
for row in image:
    for elem in row:
        if elem == "#":
            num_squares += 1

In [32]:
part2 = num_squares - num_monsters * sea_monster_size

In [33]:
print(f"Answer #2: {part2}")

Answer #2: 1705


In [34]:
# Helper function to visualize
for y in range(image_size):
    for x in range(image_size):
        print(tile_order[(x, y)], end=" ")
    print()

1453 1291 1951 2137 1997 1609 2131 1867 1187 3917 2689 2477 
3257 2659 2543 2083 2423 1511 1297 2251 3793 3221 3083 1109 
1747 1249 3517 3877 2749 2687 1999 2957 1447 3323 1933 1583 
1019 2939 1787 3541 2069 3011 1327 3539 2801 2843 2411 3407 
1483 2347 1433 2557 1709 3061 1471 1901 2267 2671 1303 3853 
1823 1801 2803 1637 3229 2063 3989 3527 2153 3719 1907 1429 
1627 2857 3359 1123 3739 3313 3023 1549 2797 2141 3191 2399 
3779 3929 2039 3769 2879 2081 2971 2503 1753 2539 3137 2017 
1663 2053 2917 3169 1103 1831 3041 3697 1987 3433 2887 3343 
1721 2777 1231 2579 2819 3709 2663 3673 1559 1031 3823 2969 
2459 1201 1063 2441 3329 3701 2707 3491 2531 2027 3347 1607 
2897 1777 3121 3371 3301 1301 2521 3463 2551 1427 1861 1439 
